# Project for Algorithms in Molecular Biology

- Mamai Maria
- Paranou Dimitra

In [1]:
import numpy as np
import random
import math
from Bio.Seq import Seq
from Bio import motifs
import operator

## GSGA

In [2]:
IUPAC = {
  'W' : ["A", "T"],
  "R" : ["A", "G"],
  "K" : ["G", "T"],
  "S" : ["G", "C"],
  "Y" : ["T", "C"],
  "M" : ["A", "C"],
  "B" : ["G", "C", "T"],
  "D" : ["G", "A", "T"],
  "H" : ["C", "A", "T"],
  "V" : ["C", "A", "G"],
  "N" : ["C", "A", "G", "T"],
}

IUPAC_bases = {
  "A": ["W", "R", "M", "D", "H", "V", "N"],
  "C": ["S", "Y", "M", "B", "H", "V", "N"],
  "G": ["R", "K", "S", "B", "D", "V", "N"],
  "T": ["W", "K", "Y", "B", "D", "H", "N"],
}

IUPAC_2bases = ["W", "R", "K", "S", "Y", "M"]
IUPAC_3bases = ["B", "D", "H", "V"]
bases = ['A', "T", "G", "C"]

b_k = 0.25

In [3]:
# convert string to list character-wise
def ConvertStringToList(string):
  list1=[]
  list1[:0]=string
  return list1

In [4]:
def hammingDistance(str1, str2):
  diffs = 0
  for ch1, ch2 in zip(str1, str2):
    if ch1 != ch2:
      diffs += 1
  return diffs

In [5]:
# Build Frequences matrix based on motifs
def BuildFrequencies(motifs):
	k = len(motifs[0])
	frequencies = [[0 for y in range(k)] for x in range(4)]
	for count in range(k):
		A=0
		C=0
		G=0
		T=0
		
		for string in motifs:
			if string[count]=='A':
				A+=1
			elif string[count]=='C':
				C+=1
			elif string[count]=='G':
				G+=1
			elif string[count]=='T':
				T+=1
			elif string[count]=='W':
				A+=0.5
				T+=0.5
			elif string[count]=='R':
				A+=0.5
				G+=0.5
			elif string[count]=='K':
				T+=0.5
				G+=0.5
			elif string[count]=='S':
				C+=0.5
				G+=0.5
			elif string[count]=='Y':
				C+=0.5
				T+=0.5
			elif string[count]=='M':
				C+=0.5
				A+=0.5
			elif string[count]=='B':
				C+=0.3
				G+=0.3
				T+=0.3
			elif string[count]=='D':
				A+=0.3
				G+=0.3
				T+=0.3
			elif string[count]=='H':
				A+=0.3
				C+=0.3
				T+=0.3
			elif string[count]=='V':
				A+=0.3
				C+=0.3
				G+=0.3
			elif string[count]=='N':
				A+=0.25
				C+=0.25
				G+=0.25
				T+=0.25
		# Insert frequencies if base A
		frequencies[0][count] = (float(A) + b_k)/(A+C+G+T+1)
		# Insert frequencies if base C
		frequencies[1][count] = (float(C) + b_k)/(A+C+G+T+1)
		# Insert frequencies if base G
		frequencies[2][count] = (float(G)+ b_k)/(A+C+G+T+1)
		# Insert frequencies if base T
		frequencies[3][count] = (float(T) + b_k)/(A+C+G+T+1)
	return frequencies

In [6]:
# Build Position Weight Matrix based on frequencies
def BuildPWM(frequencies):
  pwm = [[0 for y in range(np.shape(frequencies)[1])] for x in range(4)]
  for index1, prob in enumerate(frequencies):
    for index2, j in enumerate(prob):
       pwm[index1][index2] = math.log2(j/b_k)
  
  return pwm

In [7]:
# Evalation function that calculate the similarity between pattern string and PWM
def motif_evaluation(motif, frequencies_matrix, pwm):
  sum = 0
  for index, string in enumerate(motif):
    if string == 'A':
      sum += frequencies_matrix[0][index]*pwm[0][index]
    elif string == 'C':
      sum += frequencies_matrix[1][index]*pwm[1][index]
    elif string == 'G':
      sum += frequencies_matrix[2][index]*pwm[2][index]
    elif string == 'T':
      sum += frequencies_matrix[3][index]*pwm[3][index]
    elif string =='W':
      sum += (frequencies_matrix[0][index]*pwm[0][index] + frequencies_matrix[3][index]*pwm[3][index])/4
    elif string =='R':
      sum += (frequencies_matrix[0][index]*pwm[0][index] + frequencies_matrix[2][index]*pwm[2][index])/4
    elif string =='K':
      sum += (frequencies_matrix[2][index]*pwm[2][index] + frequencies_matrix[3][index]*pwm[3][index])/4
    elif string =='S':
      sum += (frequencies_matrix[2][index]*pwm[2][index] + frequencies_matrix[1][index]*pwm[1][index])/4
    elif string =='Y':
      sum += (frequencies_matrix[1][index]*pwm[1][index] + frequencies_matrix[3][index]*pwm[3][index])/4
    elif string =='M':
      sum += (frequencies_matrix[0][index]*pwm[0][index] + frequencies_matrix[1][index]*pwm[1][index])/4
    elif string =='B':
      sum += (frequencies_matrix[1][index]*pwm[1][index] + frequencies_matrix[2][index]*pwm[2][index] + frequencies_matrix[3][index]*pwm[3][index])/6
    elif string =='D':
      sum += (frequencies_matrix[0][index]*pwm[0][index] + frequencies_matrix[2][index]*pwm[2][index] + frequencies_matrix[3][index]*pwm[3][index])/6
    elif string =='H':
      sum += (frequencies_matrix[1][index]*pwm[1][index] + frequencies_matrix[0][index]*pwm[0][index] + frequencies_matrix[3][index]*pwm[3][index])/6
    elif string =='V':
      sum += (frequencies_matrix[1][index]*pwm[1][index] + frequencies_matrix[2][index]*pwm[2][index] + frequencies_matrix[0][index]*pwm[0][index])/6
    elif string =='N':
      sum += (frequencies_matrix[0][index]*pwm[0][index] + frequencies_matrix[1][index]*pwm[0][index] + frequencies_matrix[2][index]*pwm[0][index] + frequencies_matrix[3][index]*pwm[0][index])/8
  return sum

In [8]:
# Scoring function of possible motifs
def scoring(motif, pwm):
  sum = 0
  for index, string in enumerate(motif):
    if string == 'A':
      sum += pwm[0][index]
    elif string == 'C':
      sum += pwm[1][index]
    elif string == 'G':
      sum += pwm[2][index]
    elif string == 'T':
      sum += pwm[3][index]
    elif string =='W':
      sum += (pwm[0][index] + pwm[3][index])/4
    elif string =='R':
      sum += (pwm[0][index] + pwm[2][index])/4
    elif string =='K':
      sum += (pwm[2][index] + pwm[3][index])/4
    elif string =='S':
      sum += (pwm[2][index] + pwm[1][index])/4
    elif string =='Y':
      sum += (pwm[1][index] + pwm[3][index])/4
    elif string =='M':
      sum += (pwm[0][index] + pwm[1][index])/4
    elif string =='B':
      sum += (pwm[1][index] + pwm[2][index] + pwm[3][index])/6
    elif string =='D':
      sum += (pwm[0][index] + pwm[2][index] + pwm[3][index])/6
    elif string =='H':
      sum += (pwm[1][index] + pwm[0][index] + pwm[3][index])/6
    elif string =='V':
      sum += (pwm[1][index] + pwm[2][index] + pwm[0][index])/6
    elif string =='N':
      sum += (pwm[0][index] + pwm[0][index] + pwm[0][index] + pwm[0][index])/8
  return sum

In [9]:
# Score function that calculates the Hamming distances of motifs from the pattern/consensus string
def score(motifs):
	k = len(motifs[0])
	pattern = []
	for i in range(k):
		A=0
		C=0
		G=0
		T=0
		for string in motifs:
			if string[i]=='A':
				A+=1
			elif string[i]=='C':
				C+=1
			elif string[i]=='G':
				G+=1
			elif string[i]=='T':
				T+=1				
		if A >= C and A >= G and A >= T:
			pattern.append('A')
		elif C >= G and C >= T:
			pattern.append('C')
		elif G >= T:
			pattern.append('G')
		else:
			pattern.append('T')

	pattern = "".join(pattern)
 			
	score = 0
	for string in motifs:
		score += hammingDistance(string, pattern)
	return score

In [10]:
def next_population(motifs, dna, pwm):
  k = len(motifs[0])

  newMotifs = []
  # Calculate probilities for each k-mer in Dna_i
  for dna_string in dna:
    scores = [0 for x in range(len(dna_string)-k+1)]
    
    for i in range(len(dna_string)-k+1):
      # calculate the distances of each k-mer with
      scores[i] = scoring(dna_string[i:i+k], pwm)

    # set the threshold for selection
    threshold = -2

    for i in range(len(dna_string)-k+1):
      if scores[i] > threshold: # the higher the score, the bigger the chance to be selected
        newMotifs.append(dna_string[i:i+k])
  
  return newMotifs   

In [11]:
def selection(motifs_next_population, frequencies_matrix, pwm, len_of_motifs):
  selected_motifs = []
  scores = {}

  for motif in motifs_next_population: 
    scores[motif] = motif_evaluation(motif, frequencies_matrix, pwm)
  
  # select 2m of next population with the higher scores
  for _ in range(len_of_motifs*2):
    selected_motifs.append(max(scores, key=scores.get))
    del scores[max(scores, key=scores.get)]
    
  return selected_motifs

In [12]:
def finalSelection(dna, k, pwm):
  motifs = []

  # Calculate probilities for each k-mer in Dna_i
  for index, dna_string in enumerate(dna):
    scores = [0 for x in range(len(dna_string)-k+1)]
    for i in range(len(dna_string)-k+1):
      # calculate the score of each k-mer
      scores[i] = scoring(dna_string[i:i+k], pwm)
    # get the index of motif with the higher score
    max_index = scores.index(max(scores))
    motifs.append(Seq(dna_string[max_index:max_index+k]))

  return motifs
  

In [13]:
# crossover two parents to create two children
def crossover(p1, p2):
  # select crossover point that is not on the end of the string
  pt = random.randint(1, len(p1)-2)
  # perform crossover
  c1 = p1[:pt] + p2[pt:]
  c2 = p2[:pt] + p1[pt:]

  return [c1, c2]

In [14]:
def mutation(individual):
  individual_arr = ConvertStringToList(individual)
  # get a random position to replace a base
  ind = random.randint(0, len(individual)-1)

  # get a random position to get a char from IUPAC
  individual_arr[ind] = random.choice(list(IUPAC))

  s = ""  #remake the string from the array "pieces"
  return s.join(individual_arr) 

In [15]:
def GSGA(dna, k, N):
  motifs = []
  t = len(dna)

  # initialize random motifs
  for strand in dna:
    i = random.randrange(len(strand)-k+1)
    substr = strand[i:i+k]
    motifs.append(substr)

  # build matrices
  frequencies_matrix = BuildFrequencies(motifs)
  pwm = BuildPWM(frequencies_matrix)

  counter = 0

  for _ in range(1,N):
    motifs_next_population = next_population(motifs, dna, pwm) 
    motifs_selection = selection(motifs_next_population, frequencies_matrix, pwm, t) # select 2 * motifs length

    # create the next generation
    children = []
    for i in range(0, len(motifs_selection), 2):
      # get selected parents with mutation in pairs 
      p1, p2 = mutation(motifs_selection[i]), mutation(motifs_selection[i+1])
      child_score = {}
      # crossover
      for c in crossover(p1, p2):
        # evaluate the motif and add score to dictionary
        child_score[c] = motif_evaluation(c, frequencies_matrix, pwm)
      
      # add the individual with higher score
      children.append(max(child_score, key=child_score.get))

    # build the new position weight matrix based on the new population
    motifs = children
    frequencies_matrix = BuildFrequencies(children)
    
    # check if the pwm converges
    if abs(np.sum(np.subtract(pwm, BuildPWM(frequencies_matrix)))) < 1.2:
      counter += 1
    else:
      counter = 0
    
    pwm = BuildPWM(frequencies_matrix)

    # if pwm doesn't change for 6 consequitive times, stop the iterations
    if counter == 6:
      break
  
  # select the final motifs based on the pwm
  final_motifs = finalSelection(dna, k, pwm)
  final_scores = {}

  for motif in final_motifs:
    final_scores[motif] = scoring(motif, pwm)


  return [final_motifs, score(final_motifs), final_scores, pwm]

In [16]:
def consensus_string(motifs_arr):
  k = len(motifs_arr[0])
  consensus = []
  for i in range(k):
    A=0
    C=0
    G=0
    T=0
    for string in motifs_arr:
      if string[i]=='A':
        A+=1
      elif string[i]=='C':
        C+=1
      elif string[i]=='G':
        G+=1
      elif string[i]=='T':
        T+=1				
    if A >= C and A >= G and A >= T:
      consensus.append('A')
    elif C >= G and C >= T:
      consensus.append('C')
    elif G >= T:
      consensus.append('G')
    else:
      consensus.append('T')

  consensus = "".join(consensus)

  # create logo from motifs
  m = motifs.create(motifs_arr)
  m.weblogo("mymotif.png")
  
  return consensus

In [19]:
kmer_length, N = 8, 200
dna = ['CGCCCCTCTCGGGGGTGTTCAGTAACCGGCCA', 'GGGCGAGGTATGTGTAAGTGCCAAGGTGCCAG', 'TAGTACCGAGACCGAAAGAAGTATACAGGCGT', 'TAGATCAAGTTTCAGGTGCACGTCGGTGAACC','AATCCACCAGCTCCACGTGCAATGTTGGCCTA']
best_motifs = [None, float('inf')]

for _ in range(30):
  motifs_arr, scores, fscores, pwm = GSGA(dna, kmer_length, N)
  if scores < best_motifs[1]:
    best_motifs[0] = motifs_arr
    best_motifs[1] = scores

print(best_motifs)
print("Consensus string: " + consensus_string(best_motifs[0]))

[[Seq('TCTCGGGG'), Seq('CCAAGGTG'), Seq('TACAGGCG'), Seq('TTCAGGTG'), Seq('TCCACGTG')], 9]
Consensus string: TCCAGGTG


Solution (motifs) from the above test
- TCTCGGGG
- CCAAGGTG
- TACAGGCG
- TTCAGGTG
- TCCACGTG

We got good accuracy with parameters:
- Number of external iterations: 30
- Number of internal iterations: 200
- Threshold for choosing next population: -2
- Counts for divergent criterion: 6

### Testing

In [20]:
# randomly generate 50 datasets with length of 80 bases
sequences = []
num_dna_str = 50
len_dna_strings = 80
pattern_len = 9

for _ in range(0,num_dna_str):
  dna_string = ''
  for _ in range(0,len_dna_strings):
    rand_int = random.randint(0,3)
    dna_string += bases[rand_int]
  sequences.append(dna_string)

# randomly generate pattern
pattern = ''
for _ in range(0,pattern_len):
  rand_int = random.randint(0,3)
  pattern += bases[rand_int]

# add the mutated pattern to 12 randomly selected sequences
for _ in range(0,12):
  # select a random sequence
  seq_rand_indx = random.randint(0,num_dna_str - 1)

  # select a random position of the sequence
  pos_rand_indx = random.randint(0, len_dna_strings - pattern_len )

  # mutate pattern in one position
  pat_rand_indx = random.randint(0,pattern_len - 1)
  list1 = list(pattern)
  list1[pat_rand_indx] = bases[random.randint(0,3)]
  mutated_pattern = ''.join(list1)

  # add pattern to the sequence
  list2 = list(sequences[seq_rand_indx])
  for i in range(0,pattern_len):
    list2[pos_rand_indx + i] = mutated_pattern[i]
  sequences[seq_rand_indx] = ''.join(list2)

In [21]:
N = 200
best_motifs = [None, float('inf')]

for _ in range(30):
  motifs_arr, scores, fscores, pwm = GSGA(sequences, pattern_len, N)
  if scores < best_motifs[1]:
    best_motifs[0] = motifs_arr
    best_motifs[1] = scores

print(best_motifs)
print("Consensus string: " + consensus_string(best_motifs[0]))
print("Original motif: " + pattern)

[[Seq('AGCCTAAGC'), Seq('AGCATAACC'), Seq('AGCATAATC'), Seq('ATATTAATG'), Seq('AAAGTAAGA'), Seq('ATCTTCAGA'), Seq('AGGCTATTA'), Seq('TGCATAATA'), Seq('AACGTGAGA'), Seq('ATCTTCAGC'), Seq('AACTTAACG'), Seq('GTCTTAAGA'), Seq('ACCATAATA'), Seq('ACCCTACCA'), Seq('GGCGTCATA'), Seq('AGCGTAAGA'), Seq('AGCATAAGA'), Seq('ATTATAACG'), Seq('AGCTTTATA'), Seq('ATATTTCGA'), Seq('ATAGTAATT'), Seq('AAATTATTA'), Seq('ACGTTAAGA'), Seq('AGCATGAGA'), Seq('CGGTTAAGA'), Seq('AATTTAAGT'), Seq('ATCCTAAGG'), Seq('GCCTTAAGT'), Seq('ATAGTAGGA'), Seq('AACCGCAGA'), Seq('AGGATAAAG'), Seq('ATCAGAATA'), Seq('CTCTTAAGA'), Seq('AGCGTACGT'), Seq('AGGATGACG'), Seq('AGCATGAGA'), Seq('ATCATTAGC'), Seq('AAAGTAATA'), Seq('AAAGTATGG'), Seq('AGAGAAAGC'), Seq('ATCATAGGA'), Seq('ATCATGACA'), Seq('ATCCTACGG'), Seq('AGATTAATT'), Seq('GTAATAAGA'), Seq('GTCCTAAGA'), Seq('TCCGTAAGG'), Seq('AGATTAAGA'), Seq('AAGGTAAGG'), Seq('AGGGTAAGT')], 161]
Consensus string: AGCATAAGA
Original motif: CCAGCATGA


## Gibbs Sampling

In [22]:
def BuildProfile(motif):
	k = len(motif[0])
	profile = [[0 for y in range(k)] for x in range(4)]
	for count in range(k):
		A=0
		C=0
		G=0
		T=0
		
		# Add in Laplace counts to avoid
		# prob densities that are zero or one
		# accelerates alg runtime
		A += 1
		C += 1
		G += 1
		T += 1
		for string in motif:
			if string[count]=='A':
				A+=1
			elif string[count]=='C':
				C+=1
			elif string[count]=='G':
				G+=1
			elif string[count]=='T':
				T+=1
		# Insert frequencies if base A
		profile[0][count] = float(A)/(A+C+G+T)
		# Insert frequencies if base C
		profile[1][count] = float(C)/(A+C+G+T)
		# Insert frequencies if base G
		profile[2][count] = float(G)/(A+C+G+T)
		# Insert frequencies if base T
		profile[3][count] = float(T)/(A+C+G+T)
	return profile

In [23]:
def singleReplacementMotif(motifs, dna_i):
  k = len(motifs[0])
  profile = BuildProfile(motifs)

  # Calculate probilities for each k-mer in Dna_i
  kmerDensities = [0 for x in range(len(dna_i)-k+1)]
  for i in range(len(dna_i)-k+1):
    prob = 1
    s = 0
    for j in range(k):
      if dna_i[i+j] == 'A':
        prob *= profile[0][j]
      elif dna_i[i+j] == 'C':
        prob *= profile[1][j]
      elif dna_i[i+j] == 'G':
        prob *= profile[2][j]
      elif dna_i[i+j] == 'T':
        prob *= profile[3][j]
    kmerDensities[i] = prob

  # normalize probabilities
  normalizationTot = sum(kmerDensities)

  for i in range(len(dna_i)-k+1):
    kmerDensities[i] = kmerDensities[i]/normalizationTot
  
  # construct prefix sum for lookup		
  kmerDensities = list(accumulate(kmerDensities))

  # randomly select a k-mer
  randVal = random.random()
  
  for i in range(len(dna_i)-k+1):
    if randVal < kmerDensities[i]:
      break
  replacementKmer = dna_i[i:i+k]

  return replacementKmer

In [24]:
def accumulate(iterable, func=operator.add):
	it = iter(iterable)
	try:
		total = next(it)
	except StopIteration:
		return
	yield total
	for element in it:
		total = func(total, element)
		yield total

In [25]:
def GibbsSampler(dna, k, N):  
  Motifs = []
  t = len(dna)

  for strand in dna:
    i = random.randrange(len(strand)-k+1)
    substr = strand[i:i+k]
    Motifs.append(substr)

  bestMotifs = Motifs
  bestMotifsScore = score(bestMotifs)

  for j in range(1,N):
    i = random.randrange(t)
    subsetMotifs = Motifs[0:i]+Motifs[i+1:t]
    replacementMotif = singleReplacementMotif(subsetMotifs, dna[i])
    Motifs[i] = replacementMotif

    if score(Motifs) < bestMotifsScore:
      bestMotifs = list(Motifs)
      bestMotifsScore = score(bestMotifs)
  return [bestMotifs, bestMotifsScore]

In [28]:
kmer_length, N = 8, 200
dna = ['CGCCCCTCTCGGGGGTGTTCAGTAACCGGCCA', 'GGGCGAGGTATGTGTAAGTGCCAAGGTGCCAG', 'TAGTACCGAGACCGAAAGAAGTATACAGGCGT', 'TAGATCAAGTTTCAGGTGCACGTCGGTGAACC','AATCCACCAGCTCCACGTGCAATGTTGGCCTA']
best_motifs = [None, float('inf')]

# Repeat the Gibbs sampler search 20 times.
for repeat in range(20):
  current_motifs = GibbsSampler(dna, kmer_length, N)
  # print(current_motifs)
  if current_motifs[1] < best_motifs[1]:
      best_motifs = current_motifs
#Print the answer.
print(best_motifs)  
print("Consensus string: " + consensus_string(best_motifs[0]))  

[['TCTCGGGG', 'CCAAGGTG', 'TACAGGCG', 'TTCAGGTG', 'TCCACGTG'], 9]
Consensus string: TCCAGGTG


### Gibbs Testing

In [30]:
N = 200
best_motifs = [None, float('inf')]

for _ in range(30):
  current_motifs = GibbsSampler(sequences, pattern_len, N)
  if current_motifs[1] < best_motifs[1]:
      best_motifs = current_motifs

print(best_motifs)
print("Consensus string: " + consensus_string(best_motifs[0]))
print("Original motif: " + pattern)

[['CCATAGGTG', 'TGATCGAGC', 'GAATCCAGC', 'GCATCACGC', 'TCTTCCATC', 'CGATCTACG', 'CTAGAAAGA', 'AAATGACGG', 'CTAGAACGG', 'CCATAAGCT', 'GCATACGAC', 'TCTTAAGAA', 'TAATAACGA', 'CCCTACCAT', 'TCATATCTA', 'AGAGACAAA', 'AGATCCCAA', 'CCGTAGCGA', 'TTATAGCAA', 'ACATATCTT', 'CTGTCCCCT', 'GTCGGTCGA', 'GCATACCGC', 'CCGTCTAGA', 'AAATTGGGA', 'CGAACTCAG', 'GCATGAACA', 'CCTTAAGTG', 'CCAGACGGG', 'AGAACCGCA', 'CGTTGACAT', 'CTAGCCTTC', 'GGATCTCTT', 'CCAGAATGT', 'GCAGCCCCC', 'GCATCACTA', 'CTATATCAT', 'GCATACCGT', 'GCGTTGGAT', 'AGAGAAAGC', 'GCATATGGA', 'CAGTACCGG', 'GTATCAAGT', 'TTGTATCGG', 'ACATCTGAA', 'CGGTACAGC', 'CTATTCAAA', 'GGATAGACA', 'GAAGGTAAG', 'CCGTATCGC'], 228]
Consensus string: CCATACCGA
Original motif: CCAGCATGA
